In [1]:
cd(@__DIR__)
ENV["CELLLISTMAP_8.3_WARNING"] = "false"
include("../src/juliaEAM.jl")

using Pkg
Pkg.activate(".")

using Printf
using Unitful: Å, nm
using StaticArrays: SVector
using GLMakie
using Molly
using UnitfulAtomic
using JLD2

using PythonCall
ENV["PYTHON"] = "/SNS/users/ccu/miniconda3/envs/analysis/bin/python"

  Activating new project at `~/Documents/ABCD_J/EAM`


"/SNS/users/ccu/miniconda3/envs/analysis/bin/python"

## Load jld2 object

In [2]:
# save_object("./test_Julia_556.jld2", molly_system)
molly_system_loaded = load_object("./jld2/n_phi_5/test_Julia_556.jld2")

System with 301 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[20.227252500000002 Å, 20.2677475 Å, 42.51975 Å])

## Make movie

In [3]:
function visualize_wrap(coord_logger,
                    boundary,
                    out_filepath::AbstractString,
                    fig;
                    connections=Tuple{Int, Int}[],
                    connection_frames=[trues(length(connections)) for i in values(coord_logger)],
                    trails::Integer=0,
                    framerate::Integer=30,
                    color=:purple,
                    connection_color=:orange,
                    markersize=0.05,
                    linewidth=2.0,
                    transparency=true,
                    show_boundary::Bool=true,
                    boundary_linewidth=2.0,
                    boundary_color=:black,
                    az=1.275pi,
                    el=pi/8,
                    kwargs...)
    coords_start = first(values(coord_logger))
    dist_unit = unit(first(first(coords_start)))
    dims = n_dimensions(boundary)
    # fig = Figure()

    if dims == 3
        PointType = Point3f
        ax = Axis3(fig[1, 1], aspect=:data, azimuth=az, elevation=el)
        max_connection_dist = cbrt(box_volume(boundary)) / 2
    elseif dims == 2
        PointType = Point2f
        ax = Axis(fig[1, 1])
        ax.aspect = DataAspect()
        max_connection_dist = sqrt(box_volume(boundary)) / 2
    else
        throw(ArgumentError("found $dims dimensions but can only visualize 2 or 3 dimensions"))
    end

    positions = Observable(PointType.(ustrip_vec.(coords_start)))
    # scatter!(ax, positions; color=color, markersize=markersize, transparency=transparency,
    #             markerspace=:data, kwargs...)
    meshscatter!(ax, positions; color=color, markersize=markersize, transparency=transparency,
    kwargs...)

    if show_boundary
        lines!(
            ax,
            Molly.bounding_box_lines(boundary, dist_unit)...;
            color=boundary_color,
            linewidth=boundary_linewidth,
        )
    end

    connection_nodes = []
    for (ci, (i, j)) in enumerate(connections)
        # Don't display connected atoms that are likely connected over the box edge
        if first(connection_frames)[ci] && norm(coords_start[i] - coords_start[j]) < max_connection_dist
            if dims == 3
                push!(connection_nodes, Observable(PointType.(
                        ustrip.([coords_start[i][1], coords_start[j][1]]),
                        ustrip.([coords_start[i][2], coords_start[j][2]]),
                        ustrip.([coords_start[i][3], coords_start[j][3]]))))
            elseif dims == 2
                push!(connection_nodes, Observable(PointType.(
                        ustrip.([coords_start[i][1], coords_start[j][1]]),
                        ustrip.([coords_start[i][2], coords_start[j][2]]))))
            end
        else
            if dims == 3
                push!(connection_nodes, Observable(PointType.([0.0, 0.0], [0.0, 0.0],
                                                        [0.0, 0.0])))
            elseif dims == 2
                push!(connection_nodes, Observable(PointType.([0.0, 0.0], [0.0, 0.0])))
            end
        end
    end
    for (ci, cn) in enumerate(connection_nodes)
        lines!(ax, cn;
                color=isa(connection_color, AbstractArray) ? connection_color[ci] : connection_color,
                linewidth=isa(linewidth, AbstractArray) ? linewidth[ci] : linewidth,
                transparency=transparency)
    end

    trail_positions = []
    for trail_i in 1:trails
        push!(trail_positions, Observable(PointType.(ustrip_vec.(coords_start))))
        col = parse.(Colorant, color)
        alpha = 1 - (trail_i / (trails + 1))
        alpha_col = RGBA.(red.(col), green.(col), blue.(col), alpha)
        # scatter!(ax, trail_positions[end]; color=alpha_col,  markersize=markersize,
        #             transparency=transparency, markerspace=:data, kwargs...)
        meshscatter!(ax, trail_positions[end]; color=alpha_col, markersize=markersize, transparency=transparency,
                    kwargs...)
    end

    boundary_conv = ustrip.(dist_unit, Molly.cubic_bounding_box(boundary))
    xlims!(ax, Molly.axis_limits(boundary_conv, coord_logger, 1))
    ylims!(ax, Molly.axis_limits(boundary_conv, coord_logger, 2))
    dims == 3 && zlims!(ax, Molly.axis_limits(boundary_conv, coord_logger, 3))

    GLMakie.record(fig, out_filepath, eachindex(values(coord_logger)); framerate=framerate) do frame_i
        coords = values(coord_logger)[frame_i]
        coords = wrap_coords.(coords, (boundary,))
        u_coords = unit(coords[1][1])
        coords = [ustrip(c) for c in coords]*uconvert(u"nm", 1*u_coords)
        # print(coords[1][1])

        for (ci, (i, j)) in enumerate(connections)
            if connection_frames[frame_i][ci] && norm(coords[i] - coords[j]) < max_connection_dist
                if dims == 3
                    connection_nodes[ci][] = PointType.(
                                ustrip.([coords[i][1], coords[j][1]]),
                                ustrip.([coords[i][2], coords[j][2]]),
                                ustrip.([coords[i][3], coords[j][3]]))
                elseif dims == 2
                    connection_nodes[ci][] = PointType.(
                                ustrip.([coords[i][1], coords[j][1]]),
                                ustrip.([coords[i][2], coords[j][2]]))
                end
            else
                if dims == 3
                    connection_nodes[ci][] = PointType.([0.0, 0.0], [0.0, 0.0],
                                                        [0.0, 0.0])
                elseif dims == 2
                    connection_nodes[ci][] = PointType.([0.0, 0.0], [0.0, 0.0])
                end
            end
        end

        positions[] = PointType.(ustrip_vec.(coords))
        for (trail_i, trail_position) in enumerate(trail_positions)
            trail_position[] = PointType.(ustrip_vec.(values(coord_logger)[max(frame_i - trail_i, 1)]))
        end
    end
end


visualize_wrap (generic function with 1 method)

In [4]:
## visualize
using ColorSchemes
using Colors

# Define the color gradient
color_0 = colorant"#000000"
color_1 = colorant"#FFFFFF"
color_gradient = ColorScheme(range(color_0, color_1, length=100))

colors = []
coords_z = [c[3] for c in molly_system_loaded.coords]
z_max = maximum(coords_z[1:end-1])
z_min = minimum(coords_z[1:end-1])
for (index, value) in enumerate(molly_system_loaded.coords)
    z_component = value[3]
    z_ratio = 2*(z_component-z_min)/(z_max-z_min)-1
    color = color_gradient[z_ratio]
    push!(colors, index < length(molly_system_loaded.coords) ? color : :red)
end
framerate = 60

filepath = "videos/n_phi_5/"
# Specify boundary condition
boundary_condition = Molly.CubicBoundary([molly_system_loaded.boundary[i] for i in 1:3]...)
fig = Figure(size = (800, 800))
visualize_wrap(molly_system_loaded.loggers.coords, boundary_condition, filepath*"test_Julia_556_top.mp4", fig; 
                markersize=0.1, color=colors, az=-pi/2, el=pi/2, framerate=framerate, transparency=false)
fig = Figure(size = (800, 800))
visualize_wrap(molly_system_loaded.loggers.coords, boundary_condition, filepath*"test_Julia_556_side.mp4", fig; 
                markersize=0.1, color=colors, az=0, el=0, framerate=framerate, transparency=false)
fig = Figure(size = (800, 800))
visualize_wrap(molly_system_loaded.loggers.coords, boundary_condition, filepath*"test_Julia_556_side2.mp4", fig; 
                markersize=0.1, color=colors, az=-pi/2, el=0, framerate=framerate, transparency=false)
fig = Figure(size = (800, 800))
visualize_wrap(molly_system_loaded.loggers.coords, boundary_condition, filepath*"test_Julia_556.mp4", fig; 
                markersize=0.1, color=colors, az=-5*pi/12, framerate=framerate, transparency=false)

"videos/n_phi_5/test_Julia_556.mp4"